In [ ]:
from pandas import DataFrame as df
from pathlib import Path
import pandas as pd
from os import path
import datetime
import os

pd.options.display.float_format = "{:,.4f}".format







In [53]:
print("Starting Run:", datetime.datetime.now())

Starting Run: 2024-11-01 13:42:01.744099


In [ ]:
hedgefile_date = 20240930
hedgefile_filename = "HEDGE1001.txt"
hedgefile_path = "\\\\rgare.net\\stlfinmkts\\MarketRiskMgmt\\Pricing Requests\\2024-Orion - IUL Hedging\\ANICO Files"
hedgefile_subfldr = "Hedge Report"

# Full Path to Monthly ANICO File containing new liability info for the start of the next month
iul_hedgefile = path.join(hedgefile_path, str(hedgefile_date), hedgefile_subfldr, hedgefile_filename)

# hedge_file_input_path = '\\\\rgare.net\\stlfinmkts\\MarketRiskMgmt\\Pricing Requests\\2024-Orion - IUL Hedging\\ANICO Files\\20241031\\Hedge Report\\HEDGE1104.txt'
hedge_file_input_path = '\\\\rgare.net\\stlfinmkts\\MarketRiskMgmt\\Pricing Requests\\2024-Orion - IUL Hedging\\ANICO Files\\20241031\\Hedge Report\\HEDGE1101.txt'
iul_hedgefile = hedge_file_input_path



# Add Hedge Date Based on nxlibsubtenor formula.  All strikes are set based off closing price of underlying index on 1st BD of calendar month!
# HedgeDate = datetime.date(2024,10,1)
HedgeDate = datetime.date(2024,11,1)

# TODO:  For inforce -- Need to set TargetHedgeFactor based on historic table? 
# TODO:  For new biz -- Need to set TargetHedgeFactor based on input/param/config?
# For now assume Hedge Factor = 95%
Target_Hedge_Fctr = 0.95

iul_hedgefile

'\\\\rgare.net\\stlfinmkts\\MarketRiskMgmt\\Pricing Requests\\2024-Orion - IUL Hedging\\ANICO Files\\20241031\\Hedge Report\\HEDGE1101.txt'

In [4]:
HedgeDate.strftime('%Y%m%d')

'20241001'

In [ ]:
# GET HELPER DATAFRAMES FROM STATIC ASSUMPTIONS FOLDER
static_assum_fldr = os.path.join(os.getcwd(), 'Static_Assumptions')

from utils.assumption_loader_utils import load_static_assumptions

print("REMINDER TO UPDATE NEW BUDGET INFO in Static_Assumptions Folder!!!")

ind_to_fund_file = 'Indicator_to_FundName.csv'
coplanind_to_prod_file = 'CoPlanInd_to_Prod.csv'
fullprodind_to_budget_file = 'FullProdInd_to_Budget.csv'

print(os.path.join(static_assum_fldr, ind_to_fund_file))

ind_to_fund_df = pd.read_csv(os.path.join(static_assum_fldr, ind_to_fund_file), index_col=False)
coplanind_to_prod_df = pd.read_csv(os.path.join(static_assum_fldr, coplanind_to_prod_file), index_col=False)
fullprodind_to_budget_df = pd.read_csv(os.path.join(static_assum_fldr, fullprodind_to_budget_file), index_col=False)

# Make sure to update HedgeDate Column to a date!
fullprodind_to_budget_df['HedgeDate'] = pd.to_datetime(fullprodind_to_budget_df['HedgeDate']).dt.date
fullprodind_to_budget_df

# NOTE:  THIS WAS THE OLD APPROACH
# Filter for Budget -- NOTE:  Different Approach needed for full inforce!!!
# latest_budget_df = fullprodind_to_budget_df[(HedgeDate >= pd.to_datetime(fullprodind_to_budget_df['Start']).dt.date) 
#                                             & (HedgeDate <= pd.to_datetime(fullprodind_to_budget_df['End']).dt.date)]
# latest_budget_df = latest_budget_df[['Product_Detail', 'Indicator', 'Budget']]
# latest_budget_df
# Merge w/ Budget Info to get budget
# hedgefile_df = hedgefile_df.merge(latest_budget_df, on=['Product_Detail', 'Indicator'])

REMINDER TO UPDATE NEW BUDGET INFO in Static_Assumptions Folder!!!
\\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\Indicator_to_FundName.csv


,HedgeDate,Product_Detail,Indicator,Budget
0,2023-10-02,GIUL21,INDEX,0.0521
1,2023-10-02,GIUL21,INXFEE,0.0403
2,2023-10-02,GIUL21,MARPRT,0.0222
3,2023-10-02,GIUL21,NASIDX,0.0497
4,2023-10-02,IUL08,INDEX,0.0544
...,...,...,...,...
529,2024-11-01,IUL23,INXFEE,0.0343
530,2024-11-01,IUL23,INXHIM,0.0594
531,2024-11-01,IUL23,INXLOM,0.0594
532,2024-11-01,IUL23,MARPRT,0.0231


In [4]:
hedgefile_colnames = ['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev', 'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap', 'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier', 'Status']

hedgefile_df = pd.read_csv(iul_hedgefile, index_col=False, header=None, names=hedgefile_colnames, skipfooter=2, engine='python', low_memory=True)

# Add HedgeDate to DataFrame
hedgefile_df['HedgeDate'] = HedgeDate

# Reminder to add HedgeDate to File and to date_cols below!
date_cols = ['HedgeDate', 'IssueDate', 'Entry_Date', 'AsOf_Date']
for date_col in date_cols:
    hedgefile_df[date_col] = pd.to_datetime(hedgefile_df[date_col]).dt.date

# Strip spaces out of 'PolicyNum', 'Plan', 'Tranx' and 'Indicator columns data
hedgefile_df['PolicyNum'] = hedgefile_df['PolicyNum'].str.strip()
hedgefile_df['Plan'] = hedgefile_df['Plan'].str.strip()
hedgefile_df['Tranx'] = hedgefile_df['Tranx'].str.strip()
hedgefile_df['Indicator'] = hedgefile_df['Indicator'].str.strip()

# Filter by 'Status' containing 'R' for 'RGA' reinsured
hedgefile_df = hedgefile_df[hedgefile_df['Status'].str.contains('R')]

# Filter out 'Reversals' in 'Rev' column
hedgefile_df = hedgefile_df[hedgefile_df['Rev'].str.contains(' ')]

# Filter out 'Tranx' to only contain 'EXCHNGE TO'
hedgefile_df = hedgefile_df[hedgefile_df['Tranx'].str.contains('EXCHNGE TO')]

# Adjust Fields to Divide by 100
pct_fields = ['Floor', 'Spec_Rate', 'Spread', 'Part', 'Multiplier', 'Cap']
for fld in pct_fields:
    hedgefile_df[fld] = hedgefile_df[fld] / 100.0

# Update Cap Field for Digital Options (Based on 'Indicator' == 'INXSPC')
hedgefile_df['Cap'] = hedgefile_df.apply(lambda row: row['Spec_Rate'] if row['Indicator'] == 'INXSPC' else row['Cap'], axis=1)

# Add Strike & Adj_Liab_Ntnl to DataFrame
hedgefile_df['Strike'] = 1 + hedgefile_df['Floor'] + hedgefile_df['Spread']
hedgefile_df['Adj_Liab_Ntnl'] = Target_Hedge_Fctr * hedgefile_df['Base_Liab_Ntnl'] * hedgefile_df['Part'] * (1 + hedgefile_df['Multiplier'])

# Merge w/ coplanin_to_prod_df in order to add Product_Detail and Product to the DataFrame
hedgefile_df = hedgefile_df.merge(coplanind_to_prod_df, on=['CompID','Plan', 'Indicator'])

# Merge w/ Budget Info to get budget
hedgefile_df = hedgefile_df.merge(fullprodind_to_budget_df, on=['HedgeDate', 'Product_Detail', 'Indicator'])


C:\Users\S0053071\AppData\Local\Temp\ipykernel_45660\999492265.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  hedgefile_df = pd.read_csv(iul_hedgefile, index_col=False, header=None, names=hedgefile_colnames, skipfooter=2, engine='python', low_memory=True)


In [15]:
hedgefile_df.groupby(['Indicator', 'Cap']).agg({'Adj_Liab_Ntnl': 'sum'})

Adj_Liab_Ntnl
Indicator Cap                   
INDEX     0.0725    211,420.0585
          0.0775     93,565.3290
          0.0850    624,551.0615
          0.0925    642,272.8175
          0.0950        182.1150
          0.0975 40,456,026.6685
          0.1000  1,069,004.4385
          0.1025  4,035,673.3210
INX150    0.0575      2,336.0405
          0.0600     39,441.7580
          0.0700     65,882.2435
          0.0850  2,989,296.4670
INXFEE    9.9999 10,844,697.2885
INXHIM    0.1200  5,056,789.1392
INXLOM    0.1200  1,036,147.6078
INXSPC    0.0570      2,511.0115
          0.0600     24,830.8910
          0.0650      5,758.1020
          0.0750    333,616.7060
MARPRT    9.9999  5,628,122.3198
NASIDX    0.0925     64,461.3760
          0.1000    175,173.5495
          0.1025    609,462.5725

In [14]:
hedgefile_df.groupby(['Indicator', 'Part', 'Multiplier']).agg({'Adj_Liab_Ntnl': 'sum'})

Adj_Liab_Ntnl
Indicator Part   Multiplier                
INDEX     1.0000 0.0000     47,132,695.8095
INX150    1.0000 0.0000      3,096,956.5090
INXFEE    1.0000 0.0000     10,844,697.2885
INXHIM    1.0000 0.6000      5,056,789.1392
INXLOM    1.0000 0.2400      1,036,147.6078
INXSPC    1.0000 0.0000        366,716.7105
MARPRT    2.2000 0.0000         60,405.8906
          2.4000 0.0000      5,567,716.4292
NASIDX    1.0000 0.0000        849,097.4980

In [ ]:
hedgefile_df.groupby(['Indicator', 'Floor', 'Spread']).agg({'Adj_Liab_Ntnl': 'sum'})

Adj_Liab_Ntnl
Indicator Floor  Spread                
INDEX     0.0000 0.0000 47,132,695.8095
INX150    0.0150 0.0000  3,096,956.5090
INXFEE    0.0000 0.0775     79,271.9330
                 0.0800 10,514,755.2300
                 0.0870      1,685.6135
                 0.0900    248,984.5120
INXHIM    0.0000 0.0000  5,056,789.1392
INXLOM    0.0000 0.0000  1,036,147.6078
INXSPC    0.0000 0.0000    366,716.7105
MARPRT    0.0000 0.0000  5,628,122.3198
NASIDX    0.0000 0.0000    849,097.4980

In [7]:
required_cols = ['HedgeDate', 'CompID', 'Plan', 'Indicator', 'Product', 'Strike', 'Cap', 'Budget', 'Adj_Liab_Ntnl']
hedgefile_df_reqcols = hedgefile_df[required_cols]
hedgefile_df_reqcols
# initial_summary_df = hedgefile_df


,HedgeDate,CompID,Plan,Indicator,Product,Strike,Cap,Budget,Adj_Liab_Ntnl
0,2024-10-01,1,1U145500,INX150,IUL,1.0150,0.0850,0.0371,23.6645
1,2024-10-01,1,1U145500,INXFEE,IUL,1.0800,9.9999,0.0340,23.6740
2,2024-10-01,1,1U145500,INX150,IUL,1.0150,0.0850,0.0371,235.3055
3,2024-10-01,1,1U145500,INXFEE,IUL,1.0800,9.9999,0.0340,298.8415
4,2024-10-01,1,1U145500,INX150,IUL,1.0150,0.0850,0.0371,26.2485
...,...,...,...,...,...,...,...,...,...
178977,2024-10-01,26,1U146900,INDEX,IUL14NY,1.0000,0.0850,0.0467,79.5150
178978,2024-10-01,26,1U146900,INDEX,IUL14NY,1.0000,0.0850,0.0467,334.1815
178979,2024-10-01,26,1U146900,INXFEE,IUL14NY,1.0775,9.9999,0.0352,334.1910
178980,2024-10-01,26,1U146900,INDEX,IUL14NY,1.0000,0.0850,0.0467,383.4485


In [10]:
#hedgefile_df.groupby(['Product', 'Indicator']).agg({'Base_Liab_Ntnl':sum, 'Adj_Liab_Ntnl':sum}).to_clipboard()
initial_summary_df = hedgefile_df_reqcols.groupby(['HedgeDate', 'CompID', 'Indicator', 'Product', 'Strike', 'Cap', 'Budget']).agg({'Adj_Liab_Ntnl': 'sum'})
initial_summary_df.reset_index(inplace=True)
initial_summary_df

,HedgeDate,CompID,Indicator,Product,Strike,Cap,Budget,Adj_Liab_Ntnl
0,2024-10-01,1,INDEX,IUL,1.0000,0.0925,0.050861,4.096891e+05
1,2024-10-01,1,INDEX,IUL,1.0000,0.0975,0.051800,4.187275e+07
2,2024-10-01,1,INDEX,IUL,1.0000,0.1000,0.052800,9.084618e+05
3,2024-10-01,1,INDEX,IUL,1.0000,0.1025,0.052800,3.386709e+06
4,2024-10-01,1,INDEX,IUL14B,1.0000,0.0725,0.041100,2.103824e+05
5,2024-10-01,1,INX150,IUL,1.0150,0.0850,0.037100,1.353791e+06
6,2024-10-01,1,INX150,IUL14B,1.0150,0.0600,0.035000,3.747697e+04
7,2024-10-01,1,INXFEE,IUL,1.0800,9.9999,0.034000,1.091955e+07
8,2024-10-01,1,INXFEE,IUL,1.0900,9.9999,0.029763,1.588870e+04
9,2024-10-01,1,INXFEE,IUL14B,1.0900,9.9999,0.029800,2.272071e+05


In [11]:
liab_summary_df = initial_summary_df.merge(ind_to_fund_df, on=['Indicator'])

ordered_cols = ['HedgeDate', 'CompID', 'Indicator', 'Product', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap', 'Budget', 'Adj_Liab_Ntnl']
liab_summary_df = liab_summary_df[ordered_cols]

liab_summary_df.sort_values(by=['Fund_Name', 'CompID', 'Product'], inplace=True)
liab_summary_df.reset_index(inplace=True, drop=True)
liab_summary_df['Budget'] = liab_summary_df['Budget'].astype(float)
liab_summary_df
#liab_summary_df
# liab_summary_df.sort_values(by=['Fund_Name', 'CompID', 'Product'])

# formatted_df = liab_summary_df.style.format({
#     'Strike': '{:.2%}',  # Comma-separated thousands with no decimal places
#     'Cap': '{:.2%}',  # Percentage format with 2 decimal places
#     'Budget': '{:.2%}',  # Percentage format with 2 decimal places
#     'Adj_Liab_Ntnl': '{:,.0f}',  # Comma-separated thousands with no decimal places
# })

# print(formatted_df)



,HedgeDate,CompID,Indicator,Product,Bbg_Idx,Fund_Name,Opt_Type,Strike,Cap,Budget,Adj_Liab_Ntnl
0,2024-10-01,1,NASIDX,IUL,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.0925,0.047432,1.357278e+04
1,2024-10-01,1,NASIDX,IUL,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.1000,0.050500,1.557784e+05
2,2024-10-01,1,NASIDX,IUL,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.1025,0.051525,3.475919e+05
3,2024-10-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.0925,0.050861,4.096891e+05
4,2024-10-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.0975,0.051800,4.187275e+07
5,2024-10-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.1000,0.052800,9.084618e+05
6,2024-10-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.1025,0.052800,3.386709e+06
7,2024-10-01,1,INDEX,IUL14B,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.0725,0.041100,2.103824e+05
8,2024-10-01,26,INDEX,IUL14BNY,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.0775,0.043400,8.563528e+03
9,2024-10-01,26,INDEX,IUL14NY,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.0000,0.0850,0.046700,1.540719e+06


In [ ]:
recon_fldr = str(Path(os.getcwd()).parents[0])
# recon_file = os.path.join(recon_fldr, '20240930_LiabFile_Test.xlsx')
# liab_summary_df.to_excel(recon_file, index=True, index_label='LiabNum')
recon_file = os.path.join(recon_fldr, '20240930_LiabFile_Test.csv')
liab_summary_df.to_csv(recon_file, index=True, index_label='Liab_ID')


In [39]:
liab_summary_df.dtypes

HedgeDate         object
CompID             int64
Indicator         object
Product           object
Bbg_Idx           object
Fund_Name         object
Opt_Type          object
Strike           float64
Cap              float64
Budget           float64
Adj_Liab_Ntnl    float64
dtype: object

In [20]:
def wghtd_avg(group_df, whole_df, values, weights):
    v = whole_df.loc[group_df.index, values]
    w = whole_df.loc[group_df.index, weights]
    return (v * w).sum() / w.sum()

trading_summary_df = liab_summary_df.groupby(['HedgeDate', 'Fund_Name', 'Bbg_Idx', 'Opt_Type']).agg(
    {
        "Strike": lambda x: wghtd_avg(x, liab_summary_df, "Strike", "Adj_Liab_Ntnl"),
        "Cap": lambda x: wghtd_avg(x, liab_summary_df, "Cap", "Adj_Liab_Ntnl"),
        "Budget": lambda x: wghtd_avg(x, liab_summary_df, "Budget", "Adj_Liab_Ntnl"),
        "Adj_Liab_Ntnl": 'sum'
    }
)

trading_summary_df.reset_index(inplace=True)

trading_summary_df.rename(columns={'Cap': 'Cap/Rate', 'Adj_Liab_Ntnl': 'Notional'}, inplace=True)
trading_summary_df

,HedgeDate,Fund_Name,Bbg_Idx,Opt_Type,Strike,Cap/Rate,Budget,Notional
0,2024-10-01,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,NDX Index,Call Spread,1.000000,0.101484,0.051109,5.169431e+05
1,2024-10-01,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,SPX Index,Call Spread,1.000000,0.097344,0.051670,4.833728e+07
2,2024-10-01,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap & Floor,SPX Index,Call Spread,1.015000,0.083699,0.036752,1.452843e+06
3,2024-10-01,S&P 500 1 Yr Pt-to-Pt Multiplier w Cap,SPX Index,Call Spread,1.000000,0.120000,0.059300,5.173552e+06
4,2024-10-01,S&P 500 1 Yr Pt-to-Pt Uncapped w Fee,SPX Index,Call,1.080196,9.999900,0.033919,1.126131e+07
5,2024-10-01,S&P 500 1 Yr Specified Rate,SPX Index,Digital,1.000000,0.072703,0.051676,5.815463e+05
6,2024-10-01,S&P MARC 5% 1 Yr Pt-to-Pt Uncapped,SPMARC5P Index,Call,1.000000,9.999900,0.023643,4.724653e+06


In [19]:
trading_summary_df

In [18]:
liab_summary_df.columns

Index(['HedgeDate', 'CompID', 'Indicator', 'Product', 'Strike', 'Cap',
       'Budget', 'Adj_Liab_Ntnl', 'Orion_Idx', 'Bbg_Idx', 'Fund_Name',
       'Opt_Term_Yrs', 'Opt_Type'],
      dtype='object')

In [ ]:

# Merge w/ Budget Info to get budget
# test_df = hedgefile_df.merge(fullprodind_to_budget_df, on=['HedgeDate', 'Product_Detail', 'Indicator'])
# test_df
# hedgefile_df = hedgefile_df.merge(fullprodind_to_budget_df, on=['HedgeDate', 'Product_Detail', 'Indicator'])
hedgefile_df.columns

Index(['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev',
       'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap',
       'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge',
       'Multiplier', 'Status', 'HedgeDate', 'Strike', 'Adj_Liab_Ntnl',
       'Product_Detail', 'Product', 'Budget'],
      dtype='object')

In [ ]:


# Remove Undesired Columns
# Removed_Cols = ['Tranx', 'Rev', 'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap', 'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier', 'Status']
# desired_cols = ['CompID', 'PolicyNum', 'Plan', 'Indicator', 'Base_Liab_Ntnl', 'Part', 'Cap', 'Floor', 'Spread', 'Multiplier']


hedgefile_df




,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,Spread,Asset_Charge,Multiplier,Status,HedgeDate,Strike,Adj_Liab_Ntnl,Product_Detail,Product,Budget
0,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0000,0.0,0.0,R,2024-10-01,1.0150,23.6645,IUL14,IUL,0.0371
1,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0800,0.0,0.0,R,2024-10-01,1.0800,23.6740,IUL14,IUL,0.0343
2,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,IFRIC,2024-10-01,2024-10-01,...,0.0000,0.0,0.0,R,2024-10-01,1.0150,235.3055,IUL14,IUL,0.0371
3,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,IFRIF,2024-10-01,2024-10-01,...,0.0800,0.0,0.0,R,2024-10-01,1.0800,298.8415,IUL14,IUL,0.0343
4,1,UE000011,1U145500,2016-05-09,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0000,0.0,0.0,R,2024-10-01,1.0150,26.2485,IUL14,IUL,0.0371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178977,26,UN011777,1U146900,2023-04-14,INDEX,EXCHNGE TO,,IFRIX,2024-10-01,2024-10-01,...,0.0000,0.0,0.0,R,2024-10-01,1.0000,79.5150,IUL14NY,IUL14NY,0.0467
178978,26,UN011856,1U146900,2023-09-06,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0000,0.0,0.0,R,2024-10-01,1.0000,334.1815,IUL14NY,IUL14NY,0.0467
178979,26,UN011856,1U146900,2023-09-06,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0775,0.0,0.0,R,2024-10-01,1.0775,334.1910,IUL14NY,IUL14NY,0.0354
178980,26,UN012447,1U146900,2024-01-20,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0000,0.0,0.0,R,2024-10-01,1.0000,383.4485,IUL14NY,IUL14NY,0.0467


In [48]:
digitals_only_df = hedgefile_df[hedgefile_df['Indicator']=='INXSPC']
digital_cols = ['Part', 'Cap', 'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge',
       'Multiplier', 'Strike', 'Budget', 'Base_Liab_Ntnl', 'Adj_Liab_Ntnl']

digitals_only_df=digitals_only_df[digital_cols]
digitals_only_df['Spec_Rate'].unique()
digitals_only_df
# hedgefile_df['Tranx'].unique()

,Part,Cap,MGIR_(Cap),Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,Strike,Budget,Base_Liab_Ntnl,Adj_Liab_Ntnl
94,1.0,0.075,999.99,0.0,0.075,0.0,0.0,0.0,1.0,0.0521,187971.05,178572.4975
113,1.0,0.075,999.99,0.0,0.075,0.0,0.0,0.0,1.0,0.0521,14.94,14.1930
182,1.0,0.075,999.99,0.0,0.075,0.0,0.0,0.0,1.0,0.0521,8.60,8.1700
185,1.0,0.075,999.99,0.0,0.075,0.0,0.0,0.0,1.0,0.0521,80.25,76.2375
867,1.0,0.075,999.99,0.0,0.075,0.0,0.0,0.0,1.0,0.0521,9.42,8.9490
...,...,...,...,...,...,...,...,...,...,...,...,...
178940,1.0,0.065,999.99,0.0,0.065,0.0,0.0,0.0,1.0,0.0452,50.39,47.8705
178947,1.0,0.065,999.99,0.0,0.065,0.0,0.0,0.0,1.0,0.0452,43.58,41.4010
178960,1.0,0.065,999.99,0.0,0.065,0.0,0.0,0.0,1.0,0.0452,45.41,43.1395
178962,1.0,0.065,999.99,0.0,0.065,0.0,0.0,0.0,1.0,0.0452,49.06,46.6070


In [188]:
reordered_cols = ['HedgeDate', 'CompID', 'PolicyNum', 'Plan', 'Indicator', 'Product_Detail', 'Product', 'IssueDate', 'Tranx', 'Rev', 'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'MGIR_(Cap)', 'Spec_Rate',  'Asset_Charge', 
                  'Floor', 'Spread', 'Strike', 'Cap', 'Budget', 'Base_Liab_Ntnl', 'Multiplier', 'Part', 'Adj_Liab_Ntnl']

In [ ]:
# hedgefile_df = hedgefile_df[reordered_cols]
hedgefile_df

,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,Spread,Asset_Charge,Multiplier,Status,HedgeDate,Strike,Adj_Liab_Ntnl,Product_Detail,Product,Budget


In [187]:
hedgefile_df.columns

Index(['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev',
       'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap',
       'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge',
       'Multiplier', 'Status', 'HedgeDate', 'Strike', 'Adj_Liab_Ntnl',
       'Product_Detail', 'Product', 'Budget'],
      dtype='object')

In [179]:
recon_df = hedgefile_df[hedgefile_df['PolicyNum'].str.contains('UE246423  ')]
recon_df



,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,Base_Liab_Ntnl,Part,Cap,MGIR_(Cap),Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,Status
237474,1,UE246423,1U147800,2024-07-18,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-09-01,44935.01,100.0,10.0,3.0,0.0,0.0,0.0,0.0,0.0,


In [ ]:


# New Calculated Columns
# Strike = 1+(Floor/100) + (Spread/100)
# HedgeDate = <TBD From input/param/config!>
# OrionDate = <Confirm this is no longer needed!>
# Index = <populated based on inner join w/ assumption tbl>
# FundName = <populated based on inner join w/ assumption tbl>
# Adj_Liab_Ntnl = Base_Liab_Ntnl*Target_Hedge_Fctr*(Part/100)*(1+Multiplier/100)

In [176]:

recon_fldr = str(Path(os.getcwd()).parents[0])
recon_file = os.path.join(recon_fldr, '20240930_ReconFile.xlsx')
hedgefile_df.to_excel(recon_file, index=False)


In [148]:
hedgefile_df

,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,Spread,Asset_Charge,Multiplier,Status,HedgeDate,Strike,Adj_Liab_Ntnl,Product_Detail,Product,Budget
0,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.00,0.0,0.0,R,2024-10-01,1.0150,23.6645,IUL14,IUL,0.0371
1,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,8.00,0.0,0.0,R,2024-10-01,1.0800,23.6740,IUL14,IUL,0.0343
2,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,IFRIC,2024-10-01,2024-10-01,...,0.00,0.0,0.0,R,2024-10-01,1.0150,235.3055,IUL14,IUL,0.0371
3,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,IFRIF,2024-10-01,2024-10-01,...,8.00,0.0,0.0,R,2024-10-01,1.0800,298.8415,IUL14,IUL,0.0343
4,1,UE000011,1U145500,2016-05-09,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.00,0.0,0.0,R,2024-10-01,1.0150,26.2485,IUL14,IUL,0.0371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178977,26,UN011777,1U146900,2023-04-14,INDEX,EXCHNGE TO,,IFRIX,2024-10-01,2024-10-01,...,0.00,0.0,0.0,R,2024-10-01,1.0000,79.5150,IUL14NY,IUL14NY,0.0467
178978,26,UN011856,1U146900,2023-09-06,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.00,0.0,0.0,R,2024-10-01,1.0000,334.1815,IUL14NY,IUL14NY,0.0467
178979,26,UN011856,1U146900,2023-09-06,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,7.75,0.0,0.0,R,2024-10-01,1.0775,334.1910,IUL14NY,IUL14NY,0.0354
178980,26,UN012447,1U146900,2024-01-20,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.00,0.0,0.0,R,2024-10-01,1.0000,383.4485,IUL14NY,IUL14NY,0.0467


In [139]:
hedgefile_df['Base_Liab_Ntnl'].sum(), hedgefile_df['Adj_Liab_Ntnl'].sum()

(np.float64(71000005.46999998), np.float64(72048130.0531))

In [141]:
hedgefile_df.groupby(['Product', 'Indicator']).agg({'Base_Liab_Ntnl':sum, 'Adj_Liab_Ntnl':sum}).to_clipboard()

C:\Users\S0053071\AppData\Local\Temp\ipykernel_141868\330811211.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  hedgefile_df.groupby(['Product', 'Indicator']).agg({'Base_Liab_Ntnl':sum, 'Adj_Liab_Ntnl':sum}).to_clipboard()
C:\Users\S0053071\AppData\Local\Temp\ipykernel_141868\330811211.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  hedgefile_df.groupby(['Product', 'Indicator']).agg({'Base_Liab_Ntnl':sum, 'Adj_Liab_Ntnl':sum}).to_clipboard()


In [ ]:
# DON'T NEED TO MERGE THIS TILL THE SUMMARY PART
test_df = hedgefile_df.merge(ind_to_fund_df, on=['Indicator'])
test_df

In [129]:
# hedgefile_df['Plan'] = hedgefile_df['Plan'].str.strip()
hedgefile_df['Indicator'] = hedgefile_df['Indicator'].str.strip()
test_df = hedgefile_df.merge(coplanind_to_prod_df, on=['CompID','Plan', 'Indicator'])
test_df

,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,Spec_Rate,Spread,Asset_Charge,Multiplier,Status,HedgeDate,Strike,Adj_Liab_Ntnl,Product_Detail,Product
0,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0,0.00,0.0,0.0,R,2024-10-01,1.0150,23.6645,IUL14,IUL
1,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0,8.00,0.0,0.0,R,2024-10-01,1.0800,23.6740,IUL14,IUL
2,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,IFRIC,2024-10-01,2024-10-01,...,0.0,0.00,0.0,0.0,R,2024-10-01,1.0150,235.3055,IUL14,IUL
3,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,IFRIF,2024-10-01,2024-10-01,...,0.0,8.00,0.0,0.0,R,2024-10-01,1.0800,298.8415,IUL14,IUL
4,1,UE000011,1U145500,2016-05-09,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0,0.00,0.0,0.0,R,2024-10-01,1.0150,26.2485,IUL14,IUL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178977,26,UN011777,1U146900,2023-04-14,INDEX,EXCHNGE TO,,IFRIX,2024-10-01,2024-10-01,...,0.0,0.00,0.0,0.0,R,2024-10-01,1.0000,79.5150,IUL14NY,IUL14NY
178978,26,UN011856,1U146900,2023-09-06,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0,0.00,0.0,0.0,R,2024-10-01,1.0000,334.1815,IUL14NY,IUL14NY
178979,26,UN011856,1U146900,2023-09-06,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0,7.75,0.0,0.0,R,2024-10-01,1.0775,334.1910,IUL14NY,IUL14NY
178980,26,UN012447,1U146900,2024-01-20,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,0.0,0.00,0.0,0.0,R,2024-10-01,1.0000,383.4485,IUL14NY,IUL14NY


In [ ]:
# hedgefile_df['AsOf_Date'].drop_duplicates()
hedgefile_df['AsOf_Date'].unique()

array([datetime.date(2024, 11, 1), datetime.date(2024, 10, 1),
       datetime.date(2024, 9, 1), datetime.date(2024, 5, 1),
       datetime.date(2024, 6, 1), datetime.date(2024, 7, 1),
       datetime.date(2024, 8, 1), datetime.date(2023, 8, 1),
       datetime.date(2023, 9, 1), datetime.date(2024, 1, 1),
       datetime.date(2024, 2, 1), datetime.date(2024, 3, 1),
       datetime.date(2024, 4, 1)], dtype=object)

In [15]:
hedgefile_df['Entry_Date'].unique()

array([datetime.date(2024, 11, 1), datetime.date(2024, 10, 2),
       datetime.date(2024, 10, 18), datetime.date(2024, 10, 11),
       datetime.date(2024, 10, 7), datetime.date(2024, 10, 21),
       datetime.date(2024, 10, 14), datetime.date(2024, 10, 4),
       datetime.date(2024, 10, 22), datetime.date(2024, 10, 17),
       datetime.date(2024, 10, 3), datetime.date(2024, 10, 31),
       datetime.date(2024, 10, 29), datetime.date(2024, 10, 10),
       datetime.date(2024, 10, 9), datetime.date(2024, 10, 28),
       datetime.date(2024, 10, 15), datetime.date(2024, 10, 24),
       datetime.date(2024, 10, 23), datetime.date(2024, 10, 25),
       datetime.date(2024, 10, 8), datetime.date(2024, 10, 16),
       datetime.date(2024, 10, 30)], dtype=object)

In [19]:
hedgefile_df.groupby(['Entry_Date', 'Status', 'Rev', 'Tranx']).agg({'PolicyNum': 'count', 'Adj_Liab_Ntnl':'sum'}).to_clipboard()

In [ ]:
hedgefile_df[['AsOf_Date','Entry_Date']].drop_duplicates()

,AsOf_Date,Entry_Date
0,2024-10-01,2024-10-01
28,2024-09-05,2024-09-05
456,2024-09-11,2024-09-11
1875,2024-09-18,2024-09-18
1946,2024-09-07,2024-09-09
...,...,...
362616,2024-06-01,2024-09-05
362617,2024-07-01,2024-09-05
373449,2024-06-01,2024-09-04
373453,2024-04-01,2024-09-04


In [ ]:
# hedgefile_df['Entry_Date'].unique()
hedgefile_df['AsOf_Date'].unique(), hedgefile_df['Entry_Date'].unique()

(array([datetime.date(2024, 10, 1), datetime.date(2024, 9, 5),
        datetime.date(2024, 9, 11), datetime.date(2024, 9, 18),
        datetime.date(2024, 9, 7), datetime.date(2024, 9, 12),
        datetime.date(2024, 9, 25), datetime.date(2024, 9, 24),
        datetime.date(2024, 9, 9), datetime.date(2024, 9, 4),
        datetime.date(2024, 9, 27), datetime.date(2024, 9, 1),
        datetime.date(2024, 9, 23), datetime.date(2024, 9, 20),
        datetime.date(2024, 9, 10), datetime.date(2024, 9, 16),
        datetime.date(2024, 9, 30), datetime.date(2024, 9, 15),
        datetime.date(2024, 9, 28), datetime.date(2024, 9, 6),
        datetime.date(2024, 8, 1), datetime.date(2024, 5, 1),
        datetime.date(2024, 6, 1), datetime.date(2024, 9, 19),
        datetime.date(2024, 7, 1), datetime.date(2024, 9, 17),
        datetime.date(2024, 9, 14), datetime.date(2024, 8, 21),
        datetime.date(2024, 9, 13), datetime.date(2024, 9, 2),
        datetime.date(2024, 9, 8), datetime.date(20

In [ ]:
sorted(hedgefile_df['Indicator'].unique()), sorted(coplanind_to_prod_df['Indicator'].unique())
#coplanind_to_prod_df['Indicator'].unique()
#len(coplanind_to_prod_df['Indicator'].unique())
# len(hedgefile_df['Indicator'].unique())

# missing = []
# for plan in coplanind_to_prod_df['Plan'].unique():
#     if plan not in hedgefile_df['Plan'].unique():
#         missing.append(plan)

# missing

(['INDEX',
  'INX150',
  'INXFEE',
  'INXHIM',
  'INXLOM',
  'INXSPC',
  'MARPRT',
  'NASIDX'],
 ['INDEX',
  'INX150',
  'INXFEE',
  'INXHIM',
  'INXLOM',
  'INXSPC',
  'MARPRT',
  'NASIDX'])

In [126]:
hedgefile_df['Plan'].unique()
#coplanind_to_prod_df['Plan'].unique()
#len(coplanind_to_prod_df['Plan'].unique())
len(hedgefile_df['Plan'].unique())

missing = []
for plan in coplanind_to_prod_df['Plan'].unique():
    if plan not in hedgefile_df['Plan'].unique():
        missing.append(plan)

missing

['1U147900', '1U148000', '1U148100']

In [ ]:
hedgefile_df



,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,MGIR_(Cap),Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,Status,HedgeDate,Strike,Adj_Liab_Ntnl
2,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,3.00,1.5,0.0,0.00,0.0,0.0,R,2024-10-01,1.0150,23.6645
3,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,999.99,0.0,0.0,8.00,0.0,0.0,R,2024-10-01,1.0800,23.6740
5,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,IFRIC,2024-10-01,2024-10-01,...,3.00,1.5,0.0,0.00,0.0,0.0,R,2024-10-01,1.0150,235.3055
7,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,IFRIF,2024-10-01,2024-10-01,...,999.99,0.0,0.0,8.00,0.0,0.0,R,2024-10-01,1.0800,298.8415
9,1,UE000011,1U145500,2016-05-09,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,3.00,1.5,0.0,0.00,0.0,0.0,R,2024-10-01,1.0150,26.2485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376901,26,UN011777,1U146900,2023-04-14,INDEX,EXCHNGE TO,,IFRIX,2024-10-01,2024-10-01,...,3.00,0.0,0.0,0.00,0.0,0.0,R,2024-10-01,1.0000,79.5150
376902,26,UN011856,1U146900,2023-09-06,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,3.00,0.0,0.0,0.00,0.0,0.0,R,2024-10-01,1.0000,334.1815
376903,26,UN011856,1U146900,2023-09-06,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,999.99,0.0,0.0,7.75,0.0,0.0,R,2024-10-01,1.0775,334.1910
376904,26,UN012447,1U146900,2024-01-20,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,3.00,0.0,0.0,0.00,0.0,0.0,R,2024-10-01,1.0000,383.4485


In [108]:
coplanind_to_prod_df

,CompID,Plan,Indicator,Product_Detail,Product
0,1,1U147400,INDEX,GIUL21,IUL
1,1,1U147400,INXFEE,GIUL21,IUL
2,1,1U147400,MARPRT,GIUL21,IUL
3,1,1U147400,NASIDX,GIUL21,IUL
4,1,1U147900,INDEX,GIUL23,IUL
...,...,...,...,...,...
71,26,1U146900,INX150,IUL14NY,IUL14NY
72,26,1U145600,INXFEE,IUL14NY,IUL14NY
73,26,1U146900,INXFEE,IUL14NY,IUL14NY
74,26,1U145600,INXSPC,IUL14NY,IUL14NY


In [107]:
fullprodind_to_budget_df

,Product,Indicator,Start,End,Budget
0,GIUL21,INDEX,7/1/2021,7/31/2021,0.042107
1,GIUL21,INDEX,8/1/2021,8/31/2021,0.043673
2,GIUL21,INDEX,9/1/2021,9/30/2021,0.042776
3,GIUL21,INDEX,10/1/2021,10/31/2021,0.044768
4,GIUL21,INDEX,11/1/2021,11/30/2021,0.044626
...,...,...,...,...,...
1431,IUL23,NASIDX,6/1/2024,6/30/2024,0.050400
1432,IUL23,NASIDX,7/1/2024,7/31/2024,0.051000
1433,IUL23,NASIDX,8/1/2024,8/31/2024,0.050200
1434,IUL23,NASIDX,9/1/2024,9/30/2024,0.050500


In [106]:
ind_to_fund_df

,Indicator,Orion_Idx,Bbg_Idx,Fund_Name,Opt_Term_Yrs,Opt_Type
0,NASIDX,NASDAQ 100,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,1,Call Spread
1,INDEX,S&P 500,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,1,Call Spread
2,INX150,S&P 500,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap & Floor,1,Call Spread
3,INXHIM,S&P 500,SPX Index,S&P 500 1 Yr Pt-to-Pt Multiplier w Cap,1,Call Spread
4,INXLOM,S&P 500,SPX Index,S&P 500 1 Yr Pt-to-Pt Multiplier w Cap,1,Call Spread
5,INXFEE,S&P 500,SPX Index,S&P 500 1 Yr Pt-to-Pt Uncapped w Fee,1,Call
6,INXSPC,S&P 500,SPX Index,S&P 500 1 Yr Specified Rate,1,Digital
7,MARPRT,S&P MARC5,SPMARC5P Index,S&P MARC 5% 1 Yr Pt-to-Pt Uncapped,1,Call


In [96]:
os.path.join(os.path.pardir, 'Static_Assumptions')

'..\\Static_Assumptions'

In [100]:
os.getcwd()

'\\\\rgare.net\\stlfinmkts\\MarketRiskMgmt\\Pricing Requests\\2024-Orion - IUL Hedging\\RGA_Process\\1_Code'

In [57]:
hedgefile_df['IssueDate'].unique()

array([datetime.date(2016, 6, 23), datetime.date(2016, 5, 9),
       datetime.date(2016, 5, 20), ..., datetime.date(2017, 3, 12),
       datetime.date(2017, 5, 13), datetime.date(2024, 7, 5)],
      dtype=object)

In [ ]:


# for col, col_type in hdgfile_dtypes.items():
#     if col not in date_cols:
#         hedgefile_df[col] = hedgefile_df[col].astype(col_type)

In [ ]:
print(hedgefile_df.dtypes)


CompID              int64
PolicyNum          object
Plan               object
IssueDate          object
Indicator          object
Tranx              object
Rev                object
Dept_Desk          object
Entry_Date         object
AsOf_Date          object
Base_Liab_Ntnl    float64
Part              float64
Cap               float64
MGIR_(Cap)        float64
Floor             float64
Spec_Rate         float64
Spread            float64
Asset_Charge      float64
Multiplier        float64
Status             object
HedgeDate          object
dtype: object


In [84]:
hedgefile_df

,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,Part,Cap,MGIR_(Cap),Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,Status,HedgeDate
2,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,100.0,8.50,3.00,1.5,0.0,0.00,0.0,0.0,R,2024-10-01
3,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,100.0,999.99,999.99,0.0,0.0,8.00,0.0,0.0,R,2024-10-01
5,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,IFRIC,2024-10-01,2024-10-01,...,100.0,8.50,3.00,1.5,0.0,0.00,0.0,0.0,R,2024-10-01
7,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,IFRIF,2024-10-01,2024-10-01,...,100.0,999.99,999.99,0.0,0.0,8.00,0.0,0.0,R,2024-10-01
9,1,UE000011,1U145500,2016-05-09,INX150,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,100.0,8.50,3.00,1.5,0.0,0.00,0.0,0.0,R,2024-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376901,26,UN011777,1U146900,2023-04-14,INDEX,EXCHNGE TO,,IFRIX,2024-10-01,2024-10-01,...,100.0,8.50,3.00,0.0,0.0,0.00,0.0,0.0,R,2024-10-01
376902,26,UN011856,1U146900,2023-09-06,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,100.0,8.50,3.00,0.0,0.0,0.00,0.0,0.0,R,2024-10-01
376903,26,UN011856,1U146900,2023-09-06,INXFEE,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,100.0,999.99,999.99,0.0,0.0,7.75,0.0,0.0,R,2024-10-01
376904,26,UN012447,1U146900,2024-01-20,INDEX,EXCHNGE TO,,SWEEP,2024-10-01,2024-10-01,...,100.0,8.50,3.00,0.0,0.0,0.00,0.0,0.0,R,2024-10-01


array([datetime.date(2016, 6, 23), datetime.date(2016, 5, 9),
       datetime.date(2016, 5, 20), ..., datetime.date(2017, 3, 12),
       datetime.date(2017, 5, 13), datetime.date(2024, 7, 5)],
      dtype=object)

In [83]:
hedgefile_df = hedgefile_df[hedgefile_df['Tranx'].str.contains('EXCHNGE TO  ')]
hedgefile_df.count()

CompID            178982
PolicyNum         178982
Plan              178982
IssueDate         178982
Indicator         178982
Tranx             178982
Rev               178982
Dept_Desk         178982
Entry_Date        178982
AsOf_Date         178982
Base_Liab_Ntnl    178982
Part              178982
Cap               178982
MGIR_(Cap)        178982
Floor             178982
Spec_Rate         178982
Spread            178982
Asset_Charge      178982
Multiplier        178982
Status            178982
HedgeDate         178982
dtype: int64

In [82]:
hedgefile_df.groupby('Tranx').describe()

CompID                                                \
                 count      mean       std  min  25%  50%  75%   max   
Tranx                                                                  
ACCRUED INT        4.0  1.000000  0.000000  1.0  1.0  1.0  1.0   1.0   
CHRGE DEDUCT     166.0  1.451807  3.340400  1.0  1.0  1.0  1.0  26.0   
EXCESS INT     88364.0  1.417025  3.201848  1.0  1.0  1.0  1.0  26.0   
EXCHNGE FROM   98516.0  1.410847  3.178440  1.0  1.0  1.0  1.0  26.0   
EXCHNGE TO    178982.0  1.439849  3.286763  1.0  1.0  1.0  1.0  26.0   
INTERN SURR     1479.0  1.473293  3.408250  1.0  1.0  1.0  1.0  26.0   
SURRENDER       2434.0  1.595727  3.813694  1.0  1.0  1.0  1.0  26.0   
WDRW GRS           1.0  1.000000       NaN  1.0  1.0  1.0  1.0   1.0   
WDRW MAX         301.0  1.000000  0.000000  1.0  1.0  1.0  1.0   1.0   
WDRW NET         417.0  1.299760  2.724325  1.0  1.0  1.0  1.0  26.0   

             Base_Liab_Ntnl              ... Asset_Charge       Multiplier  \
                      count        mean  ...          75%   max      count   
Tranx                                    ...                                 
ACCRUED INT             4.0   77.867500  ...          0.0  0.00        4.0   
CHRGE DEDUCT          166.0  228.215542  ...          0.0  4.15      166.0   
EXCESS INT          88364.0   82.488738  ...          0.0  4.15    88364.0   
EXCHNGE FROM        98516.0  655.470516  ...          0.0  4.15    98516.0   
EXCHNGE TO         178982.0  396.687966  ...          0.0  4.15   178982.0   
INTERN SURR          1479.0   86.288560  ...          0.0  4.15     1479.0   
SURRENDER            2434.0  582.840222  ...          0.0  4.15     2434.0   
WDRW GRS                1.0    0.000000  ...          0.0  0.00        1.0   
WDRW MAX              301.0  554.792226  ...          0.0  2.15      301.0   
WDRW NET              417.0  596.255516  ...          0.0  4.15      417.0   

                                                             
                  mean        std  min  25%  50%  75%   max  
Tranx                                                        
ACCRUED INT   0.000000   0.000000  0.0  0.0  0.0  0.0   0.0  
CHRGE DEDUCT  0.867470   6.809567  0.0  0.0  0.0  0.0  60.0  
EXCESS INT    4.358562  14.849891  0.0  0.0  0.0  0.0  60.0  
EXCHNGE FROM  4.141337  14.509615  0.0  0.0  0.0  0.0  60.0  
EXCHNGE TO    4.308165  14.766798  0.0  0.0  0.0  0.0  60.0  
INTERN SURR   7.505071  19.028746  0.0  0.0  0.0  0.0  60.0  
SURRENDER     3.993426  14.165714  0.0  0.0  0.0  0.0  60.0  
WDRW GRS      0.000000        NaN  0.0  0.0  0.0  0.0   0.0  
WDRW MAX      0.877076   4.510895  0.0  0.0  0.0  0.0  24.0  
WDRW NET      2.906475  11.355843  0.0  0.0  0.0  0.0  60.0  

[10 rows x 80 columns]

In [81]:
hedgefile_df['Tranx'].unique()

array(['EXCESS INT  ', 'EXCHNGE TO  ', 'EXCHNGE FROM', 'SURRENDER   ',
       'INTERN SURR ', 'WDRW NET    ', 'CHRGE DEDUCT', 'WDRW MAX    ',
       'ACCRUED INT ', 'WDRW GRS    '], dtype=object)

In [ ]:

# Filter out 'Tranx' to only contain


In [60]:
# hedgefile_df['Status'] = hedgefile_df['Status'].astype(str)
hedgefile_df['Status'].unique()

array(['R', ' '], dtype=object)

In [76]:
hedgefile_df['Rev'].unique()
# hedgefile_df.groupby('Rev').describe()

array([' ', 'R'], dtype=object)

In [77]:
hedgefile_df.groupby('Rev').describe()

CompID                                               Base_Liab_Ntnl  \
        count      mean       std  min  25%  50%  75%   max          count   
Rev                                                                          
     370664.0  1.427341  3.240515  1.0  1.0  1.0  1.0  26.0       370664.0   
R      2143.0  1.256650  2.520582  1.0  1.0  1.0  1.0  26.0         2143.0   

                 ... Asset_Charge       Multiplier                            \
           mean  ...          75%   max      count      mean        std  min   
Rev              ...                                                           
     390.821482  ...          0.0  4.15   370664.0  4.280567  14.725029  0.0   
R    505.279132  ...          0.0  4.15     2143.0  2.553430  11.464141  0.0   

                          
     25%  50%  75%   max  
Rev                       
     0.0  0.0  0.0  60.0  
R    0.0  0.0  0.0  60.0  

[2 rows x 80 columns]

In [79]:

# Filter out 'Reversals' in 'Rev' column
hedgefile_df = hedgefile_df[hedgefile_df['Rev'].str.contains(' ')]

In [80]:
hedgefile_df.count()

CompID            370664
PolicyNum         370664
Plan              370664
IssueDate         370664
Indicator         370664
Tranx             370664
Rev               370664
Dept_Desk         370664
Entry_Date        370664
AsOf_Date         370664
Base_Liab_Ntnl    370664
Part              370664
Cap               370664
MGIR_(Cap)        370664
Floor             370664
Spec_Rate         370664
Spread            370664
Asset_Charge      370664
Multiplier        370664
Status            370664
HedgeDate         370664
dtype: int64

In [72]:
# Filter by 'Status' containing 'R' for 'RGA' reinsured
hedgefile_df = hedgefile_df[hedgefile_df['Status'].str.contains('R')]
hedgefile_df.count()

CompID            0
PolicyNum         0
Plan              0
IssueDate         0
Indicator         0
Tranx             0
Rev               0
Dept_Desk         0
Entry_Date        0
AsOf_Date         0
Base_Liab_Ntnl    0
Part              0
Cap               0
MGIR_(Cap)        0
Floor             0
Spec_Rate         0
Spread            0
Asset_Charge      0
Multiplier        0
Status            0
HedgeDate         0
dtype: int64

In [61]:
hedgefile_df.groupby(hedgefile_df['Status']).describe() 

CompID                                               Base_Liab_Ntnl  \
           count      mean       std  min  25%  50%  75%   max          count   
Status                                                                          
          4099.0  1.036594  0.955898  1.0  1.0  1.0  1.0  26.0         4099.0   
R       372807.0  1.426360  3.236857  1.0  1.0  1.0  1.0  26.0       372807.0   

                    ... Asset_Charge       Multiplier                       \
              mean  ...          75%   max      count      mean        std   
Status              ...                                                      
        458.824913  ...          0.0  4.15     4099.0  0.389363   4.501574   
R       391.479417  ...          0.0  4.15   372807.0  4.270639  14.708919   

                                  
        min  25%  50%  75%   max  
Status                            
        0.0  0.0  0.0  0.0  60.0  
R       0.0  0.0  0.0  0.0  60.0  

[2 rows x 80 columns]

In [ ]:
# hdgfile_dtypes = {'CompID': int, 'PolicyNum': str, 'Plan': str, 'IssueDate': datetime.date, 'Indicator': str, 'Tranx': str, 'Rev': str, 'Dept_Desk': str, 'Entry_Date':datetime.date,
#                   'AsOf_Date': datetime.date, 'Base_Liab_Ntnl': float, 'Part': float, 'Cap': float, 'MGIR_(Cap)': float, 'Floor': float, 'Spec_Rate': float, 'Spread': float, 
#                   'Asset_Charge': float, 'Multiplier': float, 'Status': str}
# hdgfile_dtypes = {'CompID': int, 'PolicyNum': str, 'Plan': str, 'IssueDate': datetime.datetime, 'Indicator': str, 'Tranx': str, 'Rev': str, 'Dept_Desk': str, 'Entry_Date':datetime.datetime,
#                   'AsOf_Date': datetime.datetime, 'Base_Liab_Ntnl': float, 'Part': float, 'Cap': float, 'MGIR_(Cap)': float, 'Floor': float, 'Spec_Rate': float, 'Spread': float, 
#                   'Asset_Charge': float, 'Multiplier': float, 'Status': str}
# hdgfile_dtypes = [int, str, str, datetime.date, str, str, str, str, datetime.date, datetime.date, float, float, float, float, float, float, float, float, float, str]

In [ ]:
# liab_file = '//rgare.net/stlfinmkts/MarketRiskMgmt/Pricing Requests/2024-Orion - IUL Hedging/RGA_Process/2_Results/20241101/iul_liab_summary.csv'
liab_file = '//rgare.net/stlfinmkts/MarketRiskMgmt/Pricing Requests/2024-Orion - IUL Hedging/RGA_Process/2_Results/20241101/iul_liab_summary.csv'
liab_summary_df = pd.read_csv(liab_file)
liab_summary_df.set_index('Liab_ID', inplace=True)


In [65]:
liab_df.dtypes

HedgeDate         object
CompID             int64
Indicator         object
Product           object
Bbg_Idx           object
Fund_Name         object
Opt_Type          object
Strike           float64
Cap              float64
Budget           float64
Adj_Liab_Ntnl    float64
dtype: object

In [ ]:
def save_liab_df(output_path, liab_summary_df):
    
    liab_df = liab_summary_df.copy(deep=True)    
    liab_df['HedgeAssetPct'] = liab_df['Adj_Liab_Ntnl'] / liab_df.groupby(['HedgeDate', 'Fund_Name'])['Adj_Liab_Ntnl'].transform('sum')
    liab_df.rename(columns={'Cap': 'Cap/Rate', 'Adj_Liab_Ntnl': 'Notional'}, inplace=True)
    liab_df.to_csv(path.join(output_path, 'iul_liab_summary.csv'), index=True, index_label='Liab_ID')

In [74]:
pd.options.display.float_format = "{:,.2f}".format

liab_df = liab_summary_df.copy(deep=True)
liab_df['HedgeAssetPct'] = liab_df['Adj_Liab_Ntnl'] / liab_df.groupby(['HedgeDate', 'Fund_Name'])['Adj_Liab_Ntnl'].transform('sum')
liab_df.rename(columns={'Cap': 'Cap/Rate', 'Adj_Liab_Ntnl': 'Notional'}, inplace=True)
liab_df

# liab_df['HedgeAsset%'] = 

,HedgeDate,CompID,Indicator,Product,Bbg_Idx,Fund_Name,Opt_Type,Strike,Cap/Rate,Budget,Notional,HedgeAssetPct
Liab_ID,,,,,,,,,,,,
0,2024-11-01,1,NASIDX,IUL,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.09,0.05,"64,510.50",0.07
1,2024-11-01,1,NASIDX,IUL,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,"179,018.24",0.21
2,2024-11-01,1,NASIDX,IUL,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,"618,191.22",0.72
3,2024-11-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.09,0.05,"649,431.95",0.01
4,2024-11-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,182.12,0.00
5,2024-11-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,"40,814,510.33",0.86
6,2024-11-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,"1,133,262.20",0.02
7,2024-11-01,1,INDEX,IUL,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,"4,156,548.83",0.09
8,2024-11-01,1,INDEX,IUL14B,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.07,0.04,"215,518.41",0.00


In [ ]:
def wghtd_avg(group_df, whole_df, values, weights):
    v = whole_df.loc[group_df.index, values]
    w = whole_df.loc[group_df.index, weights]
    return (v * w).sum() / w.sum()

trading_summary_df = liab_summary_df.groupby(['HedgeDate', 'Fund_Name', 'Bbg_Idx', 'Opt_Type']).agg(
    {
        "Strike": lambda x: wghtd_avg(x, liab_summary_df, "Strike", "Adj_Liab_Ntnl"),
        "Cap": lambda x: wghtd_avg(x, liab_summary_df, "Cap", "Adj_Liab_Ntnl"),
        "Budget": lambda x: wghtd_avg(x, liab_summary_df, "Budget", "Adj_Liab_Ntnl"),
        "Adj_Liab_Ntnl": 'sum'
    }
)

trading_summary_df.reset_index(inplace=True)

trading_summary_df.rename(columns={'Cap': 'Cap/Rate', 'Adj_Liab_Ntnl': 'Notional'}, inplace=True)
trading_summary_df

In [75]:
liab_df.to_clipboard()

In [16]:
from mthly_hedge_file import HedgeFile
%load_ext autoreload
%autoreload 2

In [98]:
hdgfile=HedgeFile()

Reading in Helper Dataframes (used for static lookups to obtain values for related fields)
REMINDER TO UPDATE NEW BUDGET INFO in Static_Assumptions Folder for ProductDetailsByHedgeDate.csv!!!
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\HedgeDates.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\HdgFctrLU.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\CoPlanInd_to_Prod.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\Indicator_to_FundName.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\ProductDetailsByHedgeDate.csv
Converting Static Ass

\\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\mthly_hedge_file.py:133: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  hedgefile_df = pd.read_csv(self.hedge_file, index_col=False, header=None, names=hedgefile_colnames, skipfooter=2, engine='python', low_memory=True)


Time Reading Hedge File: 2 mins 14 secs
Merging Hedge File w/ Helper Files to Add Additional Fields
Finished...Time Merging Hedge File: 0 mins 1 secs


In [32]:
hdgfile.hedgefile_df.dtypes

CompID              int64
PolicyNum          object
Plan               object
IssueDate          object
Indicator          object
Tranx              object
Rev                object
Dept_Desk          object
Entry_Date         object
AsOf_Date          object
Base_Liab_Ntnl    float64
Part              float64
Cap               float64
MGIR_(Cap)        float64
Floor             float64
Spec_Rate         float64
Spread            float64
Asset_Charge      float64
Multiplier        float64
Status             object
HedgeDate          object
dtype: object

In [47]:
test_dt = datetime.date(2024,2,1)

# hdg_fctr_df = hdgfile.assum_dfs['HdgFctrLU']
# hdg_fctr_df.dtypes
# hdgfile.get_hdg_fctr(test_dt, 1)
# hdgfile.get_hdg_fctr(test_dt, 1).values[0]
hdgfile.get_hdg_fctr(test_dt, 26)

np.float64(0.776)

In [33]:
hdgfile.assum_dfs = hdgfile.get_static_assum_dfs()

Reading in Helper Dataframes (used for static lookups to obtain values for related fields)
REMINDER TO UPDATE NEW BUDGET INFO in Static_Assumptions Folder for ProductDetailsByHedgeDate.csv!!!
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\HedgeDates.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\HdgFctrLU.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\CoPlanInd_to_Prod.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\Indicator_to_FundName.csv
Reading Data from \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\Static_Assumptions\ProductDetailsByHedgeDate.csv
Converting Static Ass

In [ ]:
test_df = hdgfile.assum_dfs['HedgeDates']
test_df['HedgeDate']

pandas.core.series.Series

In [48]:
hdgfile.hedgefile_df['HedgeFactor'] = hdgfile.hedgefile_df.apply(lambda row: hdgfile.get_hdg_fctr(row['HedgeDate'], row['CompID']), axis=1)
hdgfile.hedgefile_df['HedgeFactor']

2        0.9500
3        0.9500
5        0.9500
7        0.9500
9        0.9500
          ...  
383133   0.7600
383134   0.7600
383135   0.7600
383136   0.7600
383137   0.7600
Name: HedgeFactor, Length: 180909, dtype: float64

In [61]:
hdg_fctr_df = hdgfile.assum_dfs['HdgFctrLU']
# hdg_fctr_df.set_index('HedgeDate', inplace=True)
test_dict = hdg_fctr_df.to_dict(orient='index')
test_dict

new_dict = {}
for dt in test_dict.keys():
    new_dict[(dt, 1)] = test_dict[dt]['HedgeRatio'] * test_dict[dt]['Coins']
    new_dict[(dt, 26)] = test_dict[dt]['HedgeRatio_NY'] * test_dict[dt]['Coins_NY']

print(new_dict)
# hdg_fctr_df

{(datetime.date(2023, 7, 3), 1): 0.97, (datetime.date(2023, 7, 3), 26): 0.776, (datetime.date(2023, 8, 1), 1): 0.97, (datetime.date(2023, 8, 1), 26): 0.776, (datetime.date(2023, 9, 1), 1): 0.97, (datetime.date(2023, 9, 1), 26): 0.776, (datetime.date(2023, 10, 2), 1): 0.97, (datetime.date(2023, 10, 2), 26): 0.776, (datetime.date(2023, 11, 1), 1): 0.97, (datetime.date(2023, 11, 1), 26): 0.776, (datetime.date(2023, 12, 1), 1): 0.97, (datetime.date(2023, 12, 1), 26): 0.776, (datetime.date(2024, 1, 2), 1): 0.97, (datetime.date(2024, 1, 2), 26): 0.776, (datetime.date(2024, 2, 1), 1): 0.97, (datetime.date(2024, 2, 1), 26): 0.776, (datetime.date(2024, 3, 1), 1): 0.97, (datetime.date(2024, 3, 1), 26): 0.776, (datetime.date(2024, 4, 1), 1): 0.97, (datetime.date(2024, 4, 1), 26): 0.776, (datetime.date(2024, 5, 1), 1): 0.97, (datetime.date(2024, 5, 1), 26): 0.776, (datetime.date(2024, 6, 3), 1): 0.97, (datetime.date(2024, 6, 3), 26): 0.776, (datetime.date(2024, 7, 1), 1): 0.97, (datetime.date(2024

In [63]:
hf_dict = hdgfile.create_hdg_fctr_dict()

In [ ]:
hdgfile.hedgefile_df['HdgFctr'] = hdgfile.hedgefile_df.apply(lambda row: hf_dict[(row['HedgeDate'], row['CompID'])], axis=1)
hdgfile.hedgefile_df['HdgFctr']

2        0.9500
3        0.9500
5        0.9500
7        0.9500
9        0.9500
          ...  
383133   0.7600
383134   0.7600
383135   0.7600
383136   0.7600
383137   0.7600
Name: HedgeFactor, Length: 180909, dtype: float64

In [75]:
hdgfile.hdgfctr_dict = hdgfile.create_hdg_fctr_dict()

In [77]:
print(hdgfile.hdgfctr_dict)

{(datetime.date(2023, 7, 3), 1): 0.97, (datetime.date(2023, 7, 3), 26): 0.776, (datetime.date(2023, 8, 1), 1): 0.97, (datetime.date(2023, 8, 1), 26): 0.776, (datetime.date(2023, 9, 1), 1): 0.97, (datetime.date(2023, 9, 1), 26): 0.776, (datetime.date(2023, 10, 2), 1): 0.97, (datetime.date(2023, 10, 2), 26): 0.776, (datetime.date(2023, 11, 1), 1): 0.97, (datetime.date(2023, 11, 1), 26): 0.776, (datetime.date(2023, 12, 1), 1): 0.97, (datetime.date(2023, 12, 1), 26): 0.776, (datetime.date(2024, 1, 2), 1): 0.97, (datetime.date(2024, 1, 2), 26): 0.776, (datetime.date(2024, 2, 1), 1): 0.97, (datetime.date(2024, 2, 1), 26): 0.776, (datetime.date(2024, 3, 1), 1): 0.97, (datetime.date(2024, 3, 1), 26): 0.776, (datetime.date(2024, 4, 1), 1): 0.97, (datetime.date(2024, 4, 1), 26): 0.776, (datetime.date(2024, 5, 1), 1): 0.97, (datetime.date(2024, 5, 1), 26): 0.776, (datetime.date(2024, 6, 3), 1): 0.97, (datetime.date(2024, 6, 3), 26): 0.776, (datetime.date(2024, 7, 1), 1): 0.97, (datetime.date(2024

In [78]:
hdgfile.hdgfctr_dict[(datetime.date(2024, 11, 1), 1)]

0.95

In [81]:
budget_df = hdgfile.get_budget_df()
budget_df['Product_Detail'].unique()

array(['IUL08', 'IUL14', 'IUL14B', 'IUL19', 'IUL21', 'IUL23', 'GIUL21',
       'IUL14NY', 'IUL14BNY', 'GIUL23'], dtype=object)

In [83]:
# hdgfile.hedgefile_df['Product_Detail'].unique()
hdgfile.hedgefile_df.columns

Index(['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev',
       'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap',
       'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge',
       'Multiplier', 'Status', 'HedgeDate', 'HedgeFactor', 'HdgFctr', 'Strike',
       'Adj_Liab_Ntnl', 'Product_Detail_x', 'Product_x', 'Budget',
       'Product_Detail_y', 'Product_y'],
      dtype='object')

In [79]:
hdgfile.merge_dataframes()

Merging Hedge File w/ Helper Files to Add Additional Fields


KeyError: 'Product_Detail'

In [99]:
hdgfile.hedgefile_df

,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,Asset_Charge,Multiplier,Status,HedgeDate,HdgFctr,Strike,Adj_Liab_Ntnl,Product_Detail,Product,Budget
0,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0150,23.6075,IUL14,IUL,0.0383
1,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0800,23.6075,IUL14,IUL,0.0360
2,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,IFRIC,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0150,238.2885,IUL14,IUL,0.0383
3,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,IFRIF,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0800,288.9710,IUL14,IUL,0.0360
4,1,UE000011,1U145500,2016-05-09,INX150,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0150,26.2580,IUL14,IUL,0.0383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180904,26,UN011777,1U146900,2023-04-14,INDEX,EXCHNGE TO,,IFRIX,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.7600,1.0000,63.6348,IUL14NY,IUL14NY,0.0482
180905,26,UN011856,1U146900,2023-09-06,INDEX,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.7600,1.0000,276.0320,IUL14NY,IUL14NY,0.0482
180906,26,UN011856,1U146900,2023-09-06,INXFEE,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.7600,1.0775,276.0320,IUL14NY,IUL14NY,0.0372
180907,26,UN012447,1U146900,2024-01-20,INDEX,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.7600,1.0000,306.8652,IUL14NY,IUL14NY,0.0482


In [70]:
hdgfile.hedgefile_df['Adj_Liab_Ntnl'].sum()

np.float64(74848930.3986)

In [71]:
int('01')

1

In [84]:
hdgfile.hedgefile_df = hdgfile.import_mthly_hedge_txtfile()

Importing Hedge Txt File


\\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\mthly_hedge_file.py:133: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  hedgefile_df = pd.read_csv(self.hedge_file, index_col=False, header=None, names=hedgefile_colnames, skipfooter=2, engine='python', low_memory=True)


Time Reading Hedge File: 0 mins 35 secs


In [85]:
hdgfile.hedgefile_df

,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,Part,Cap,MGIR_(Cap),Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,Status,HedgeDate
2,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,1.0000,0.0850,3.0000,0.0150,0.0000,0.0000,0.0000,0.0000,R,2024-11-01
3,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,1.0000,9.9999,999.9900,0.0000,0.0000,0.0800,0.0000,0.0000,R,2024-11-01
5,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,IFRIC,2024-11-01,2024-11-01,...,1.0000,0.0850,3.0000,0.0150,0.0000,0.0000,0.0000,0.0000,R,2024-11-01
7,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,IFRIF,2024-11-01,2024-11-01,...,1.0000,9.9999,999.9900,0.0000,0.0000,0.0800,0.0000,0.0000,R,2024-11-01
9,1,UE000011,1U145500,2016-05-09,INX150,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,1.0000,0.0850,3.0000,0.0150,0.0000,0.0000,0.0000,0.0000,R,2024-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383133,26,UN011777,1U146900,2023-04-14,INDEX,EXCHNGE TO,,IFRIX,2024-11-01,2024-11-01,...,1.0000,0.0850,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,R,2024-11-01
383134,26,UN011856,1U146900,2023-09-06,INDEX,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,1.0000,0.0850,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,R,2024-11-01
383135,26,UN011856,1U146900,2023-09-06,INXFEE,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,1.0000,9.9999,999.9900,0.0000,0.0000,0.0775,0.0000,0.0000,R,2024-11-01
383136,26,UN012447,1U146900,2024-01-20,INDEX,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,1.0000,0.0850,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,R,2024-11-01


In [86]:
hdgfile.hedgefile_df['HdgFctr'] = hdgfile.hedgefile_df.apply(lambda row: hdgfile.hdgfctr_dict[(row['HedgeDate'], row['CompID'])], axis=1)

In [89]:
# Add Strike & Adj_Liab_Ntnl to DataFrame
hdgfile.hedgefile_df['Strike'] = 1 + hdgfile.hedgefile_df['Floor'] + hdgfile.hedgefile_df['Spread']
hdgfile.hedgefile_df['Adj_Liab_Ntnl'] = hdgfile.hedgefile_df['HdgFctr'] * hdgfile.hedgefile_df['Base_Liab_Ntnl'] * hdgfile.hedgefile_df['Part'] * (1 + hdgfile.hedgefile_df['Multiplier'])


hdgfile.hedgefile_df
hdgfile.hedgefile_df.columns

Index(['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev',
       'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap',
       'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge',
       'Multiplier', 'Status', 'HedgeDate', 'HdgFctr', 'Strike',
       'Adj_Liab_Ntnl'],
      dtype='object')

In [90]:
# Merge w/ coplanin_to_prod_df in order to add Product_Detail and Product to the DataFrame
hdgfile.hedgefile_df = hdgfile.hedgefile_df.merge(hdgfile.assum_dfs['CoPlanInd_to_Prod'], on=['CompID','Plan', 'Indicator'])
hdgfile.hedgefile_df.columns

Index(['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev',
       'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap',
       'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge',
       'Multiplier', 'Status', 'HedgeDate', 'HdgFctr', 'Strike',
       'Adj_Liab_Ntnl', 'Product_Detail', 'Product'],
      dtype='object')

In [91]:
hdgfile.hedgefile_df['Product_Detail'].unique()

array(['IUL14', 'IUL14B', 'IUL19', 'IUL21', 'GIUL21', 'IUL23', 'IUL08',
       'GIUL23', 'IUL14NY', 'IUL14BNY'], dtype=object)

In [94]:
test_df = hdgfile.hedgefile_df.merge(hdgfile.get_budget_df(), on=['HedgeDate', 'Product_Detail', 'Indicator'])
# budget_df = hdgfile.get_budget_df()
# budget_df.columns
test_df

,CompID,PolicyNum,Plan,IssueDate,Indicator,Tranx,Rev,Dept_Desk,Entry_Date,AsOf_Date,...,Asset_Charge,Multiplier,Status,HedgeDate,HdgFctr,Strike,Adj_Liab_Ntnl,Product_Detail,Product,Budget
0,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0150,23.6075,IUL14,IUL,0.0383
1,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0800,23.6075,IUL14,IUL,0.0360
2,1,UE000006,1U145500,2016-06-23,INX150,EXCHNGE TO,,IFRIC,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0150,238.2885,IUL14,IUL,0.0383
3,1,UE000006,1U145500,2016-06-23,INXFEE,EXCHNGE TO,,IFRIF,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0800,288.9710,IUL14,IUL,0.0360
4,1,UE000011,1U145500,2016-05-09,INX150,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.9500,1.0150,26.2580,IUL14,IUL,0.0383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180904,26,UN011777,1U146900,2023-04-14,INDEX,EXCHNGE TO,,IFRIX,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.7600,1.0000,63.6348,IUL14NY,IUL14NY,0.0482
180905,26,UN011856,1U146900,2023-09-06,INDEX,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.7600,1.0000,276.0320,IUL14NY,IUL14NY,0.0482
180906,26,UN011856,1U146900,2023-09-06,INXFEE,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.7600,1.0775,276.0320,IUL14NY,IUL14NY,0.0372
180907,26,UN012447,1U146900,2024-01-20,INDEX,EXCHNGE TO,,SWEEP,2024-11-01,2024-11-01,...,0.0000,0.0000,R,2024-11-01,0.7600,1.0000,306.8652,IUL14NY,IUL14NY,0.0482


In [96]:
print(hdgfile.hdgfctr_dict)

{(datetime.date(2023, 7, 3), 1): 0.97, (datetime.date(2023, 7, 3), 26): 0.776, (datetime.date(2023, 8, 1), 1): 0.97, (datetime.date(2023, 8, 1), 26): 0.776, (datetime.date(2023, 9, 1), 1): 0.97, (datetime.date(2023, 9, 1), 26): 0.776, (datetime.date(2023, 10, 2), 1): 0.97, (datetime.date(2023, 10, 2), 26): 0.776, (datetime.date(2023, 11, 1), 1): 0.97, (datetime.date(2023, 11, 1), 26): 0.776, (datetime.date(2023, 12, 1), 1): 0.97, (datetime.date(2023, 12, 1), 26): 0.776, (datetime.date(2024, 1, 2), 1): 0.97, (datetime.date(2024, 1, 2), 26): 0.776, (datetime.date(2024, 2, 1), 1): 0.97, (datetime.date(2024, 2, 1), 26): 0.776, (datetime.date(2024, 3, 1), 1): 0.97, (datetime.date(2024, 3, 1), 26): 0.776, (datetime.date(2024, 4, 1), 1): 0.97, (datetime.date(2024, 4, 1), 26): 0.776, (datetime.date(2024, 5, 1), 1): 0.97, (datetime.date(2024, 5, 1), 26): 0.776, (datetime.date(2024, 6, 3), 1): 0.97, (datetime.date(2024, 6, 3), 26): 0.776, (datetime.date(2024, 7, 1), 1): 0.97, (datetime.date(2024

In [101]:
hdgfile.hedgefile_df.columns

Index(['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev',
       'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap',
       'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge',
       'Multiplier', 'Status', 'HedgeDate', 'HdgFctr', 'Strike',
       'Adj_Liab_Ntnl', 'Product_Detail', 'Product', 'Budget'],
      dtype='object')

In [102]:
hdgfile.hedgefile_df.describe()

,CompID,Base_Liab_Ntnl,Part,Cap,MGIR_(Cap),Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,HdgFctr,Strike,Adj_Liab_Ntnl,Budget
count,"180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000","180,909.0000"
mean,1.4461,404.5807,1.0660,2.7592,283.8021,0.0005,0.0010,0.0177,0.3083,0.0428,0.9466,1.0182,413.7380,0.0484
std,3.3095,"8,299.3329",0.2965,4.3883,448.4509,0.0026,0.0083,0.0333,1.0376,0.1472,0.0252,0.0331,"8,227.3037",0.0100
min,1.0000,0.0100,1.0000,0.0570,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7600,1.0000,0.0095,0.0236
25%,1.0000,24.0200,1.0000,0.0975,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9500,1.0000,24.2630,0.0360
50%,1.0000,66.6000,1.0000,0.1000,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9500,1.0000,68.5140,0.0535
75%,1.0000,201.1700,1.0000,9.9999,999.9900,0.0000,0.0000,0.0000,0.0000,0.0000,0.9500,1.0150,208.8195,0.0535
max,26.0000,"2,602,643.1200",2.4000,9.9999,999.9900,0.0150,0.0750,0.0900,4.1500,0.6000,0.9500,1.0900,"2,472,510.9640",0.0612


In [109]:
desired_cols = ['HedgeDate', 'CompID', 'PolicyNum', 'Plan', 'Indicator', 'Part', 'Cap', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier', 'Strike', 'Budget', 'HdgFctr', 'Base_Liab_Ntnl', 'Adj_Liab_Ntnl']
group_by_cols = ['HedgeDate', 'CompID', 'PolicyNum', 'Plan', 'Indicator', 'Part', 'Cap', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier', 'Strike', 'Budget', 'HdgFctr']

hedgefile_inforce_summary_df = hdgfile.hedgefile_df.groupby(group_by_cols).agg({'Base_Liab_Ntnl': 'sum', 'Adj_Liab_Ntnl': 'sum'})
hedgefile_inforce_summary_df.reset_index(inplace=True)


hedgefile_inforce_summary_df
# hedgefile_inforce_summary_df.rename(columns={'Cap': 'Cap/Rate', 'Adj_Liab_Ntnl': 'Notional'}, inplace=True)
# hdgfile.hedgefile_df.columns
# hedgefile_inforce_summary_df.columns


,HedgeDate,CompID,PolicyNum,Plan,Indicator,Part,Cap,Floor,Spec_Rate,Spread,Asset_Charge,Multiplier,Strike,Budget,HdgFctr,Base_Liab_Ntnl,Adj_Liab_Ntnl
0,2024-11-01,1,U0600960,1U144600,INDEX,1.0000,0.0975,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0535,0.9500,"4,965.5800","4,717.3010"
1,2024-11-01,1,U0600961,1U144600,INDEX,1.0000,0.0975,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0535,0.9500,"3,232.9900","3,071.3405"
2,2024-11-01,1,U0600963,1U144600,INDEX,1.0000,0.0975,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0535,0.9500,"1,378.5900","1,309.6605"
3,2024-11-01,1,U0600972,1U144600,INDEX,1.0000,0.0975,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0535,0.9500,"1,155.1800","1,097.4210"
4,2024-11-01,1,U0600979,1U144600,INDEX,1.0000,0.0975,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0535,0.9500,"2,033.4600","1,931.7870"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101273,2024-11-01,26,UNS03024,1U146900,INX150,1.0000,0.0700,0.0150,0.0000,0.0000,0.0000,0.0000,1.0150,0.0314,0.7600,5.4500,4.1420
101274,2024-11-01,26,UNS03037,1U146900,INDEX,1.0000,0.0850,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0482,0.7600,49.7000,37.7720
101275,2024-11-01,26,UNS03037,1U146900,INX150,1.0000,0.0700,0.0150,0.0000,0.0000,0.0000,0.0000,1.0150,0.0314,0.7600,49.7000,37.7720
101276,2024-11-01,26,UNS03055,1U146900,INDEX,1.0000,0.0850,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0482,0.7600,65.7600,49.9776


In [104]:
hedgefile_inforce_summary_df.describe()

,Base_Liab_Ntnl,Adj_Liab_Ntnl
count,"101,278.0000","101,278.0000"
mean,722.6869,739.0443
std,"11,501.6555","11,468.3386"
min,0.0200,0.0190
25%,56.2800,58.0545
50%,157.4400,162.9393
75%,423.3325,433.5752
max,"2,701,326.6600","2,566,260.3270"


In [110]:
test_file = 'iul_final_detail.xlsx'
test_file.split('.')[1]

'xlsx'